In [ ]:
import argparse
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import skimage
import yaml
from compartmentalizer.create_compartment import run_morpho_analysis
from matplotlib import pyplot as plt
from phenoscapes.cli import process_sample, read_config
from phenoscapes.feature_extraction import extract_features
from phenoscapes.sc import convert_to_h5ad, plot_summary
from phenoscapes.segmentation import generate_label_overlay, run_cellpose
from phenoscapes.utils import get_metadata
from skimage import io
from skimage.measure import regionprops
from tqdm import tqdm

In [ ]:
os.chdir("/cluster/project/treutlein/DATA/imaging/4i_Data/Brain_ECM_4i_2_v2/")
dir_output = os.getcwd()
dir_bg_subtracted = Path(dir_output, "bg_subtracted")
config = read_config(f"{dir_output}/config.yaml")

df_stains = pd.read_csv("stain_metadata.csv")

In [ ]:
samples = [  # Day 7
    "R066_0",
    "R062_0",
    "R073_0",
    "R072_0",
    # Day 15
    #'R021_0',
    "R026_0",
    "R022_0",
    "R024_0",
    "R049_0",
    "R055_0",
    "R044_0",
    "R057_0",
    # Day 21
    "R076_1",
    "R087_0",
    "R082_0",
    "R108_1",
    "R112_1",
    "R113_1",
    "R115_0",
]

In [ ]:
# merge nuclei and cytoplasm into cell mask
dir_segmented = Path(dir_output, "segmented_cell_nuclei")
dir_segmented_cyto = Path(dir_output, "segmented_cytoplasma")
dir_segmented_cell = Path(dir_output, "segmented_cell")
dir_segmented_cell.mkdir(parents=True, exist_ok=True)

for sample in tqdm(samples):
    mask = io.imread(Path(dir_segmented, sample + ".tif")) + io.imread(
        Path(dir_segmented_cyto, sample + ".tif")
    )
    io.imsave(Path(dir_segmented_cell, sample + ".tif"), mask.astype(np.uint16))

In [ ]:
from joblib import Parallel, delayed

run = Parallel(n_jobs=2, backend="multiprocessing", verbose=20)(
    delayed(run_morpho_analysis)(sample, config) for sample in reversed(samples)
)